##### Branch out to new workspace notebook - post activity

After cloning a workspace, this notebook will reconfigure any references to the old workspace by rebinding them to the new workspace. 

For example a pipeline referencing a warehouse or a default lakehouse of a notebook.

This notebook runs post activity tasks can be run after [branch out to new workspace functionality](https://blog.fabric.microsoft.com/en-us/blog/introducing-new-branching-capabilities-in-fabric-git-integration) or the [custom AzDO script](https://github.com/microsoft/fabric-toolbox/blob/main/accelerators/CICD/Branch-out-to-new-workspace/AzDO/scripts/BranchOut-Feature-Workspace-Automation.py).

Summary of post activities in order:
<ul>
<li>Default lakehouses and warehouse are updated to local lakehouse/warehouses</li>
<li>Either creates shortcuts in local lakehouse back to tables in the source lakehouse, or copies the data from source lakehouse. Set via parameter below.</li>
<li>Copy warehouse data. Set via parameter below</li>
<li>Changes directlake semantic model connections for semantic models to "local" lakehouse/warehouse</li> 
<li>Rebinds reports to "local" semantic models</li>
<li>Changes pipeline lakehouse/warehouse references to local item</li>
<li>Ability to swap connections in pipelines from old to new</li>
<li>Commit changes to git</li>
</ul>

Requirements:
<ul>
<li>Requires Semantic Link Labs installed by pip install below or added to environment library.</li>
<li>Requires JmesPath library for data pipeline JSON manipulation i.e. connection swaps.</li>
</ul>

Limitations of current script:

<ul>
<li>Does not recreate item shares or external shortcuts</li>
<li>Does not re-apply lakehouse SQL Endpoint or Warehouse object/row/column level security</li>
<li>Does not recreate data access roles in Lakehouse</li>
<li>Untested with Lakehouses where with schema support enabled</li>
</ul>




##### Install semantic link labs to support advanced functionality

https://semantic-link-labs.readthedocs.io/en/latest/index.html
https://github.com/microsoft/semantic-link-labs/blob/main/README.md



In [ ]:
%pip -q install semantic-link-labs


##### Install Jmespath to make data pipeline changes such as updating linked notebooks, warehouses and lakehouses 

In [ ]:
%pip install jmespath

##### Set these parameters if running as a standaone noteook
Before running this notebook ensure these parameters are set correctly. If necessary these can be passed in via a data factory pipeline.

If running this notebook from Azure DevOps ignore these parameters as these will be passed from the release pipeline script

In [ ]:
source_ws = ''
target_ws = ''

# Either copy lakehouse data or create shortcuts, set at most one of these to True 
copy_lakehouse_data = False
create_lakehouse_shortcuts = True

# Option to copy warehouse data if required
copy_warehouse_data = False

# If false then shortcuts will be created. If you wish to create shortcuts based on a pattern match please set the param below
# enter pattern match for creating shortcuts - see https://github.com/arasdk/fabric-code-samples/blob/main/shortcuts/fabric_shortcut_creator.py 
PATTERN_MATCH = ["*"]

# Set connections to be replaced from previous name or ID to new name or ID.
connections_from_to = () #('https://api.fabric.microsoft.com/v1/workspaces/ admin','4498340c-27cf-4c6e-a025-00e5de6b0726'),('4498340c-27cf-4c6e-a025-00e5de6b0726','https://api.fabric.microsoft.com/v1/workspaces/ admin'),('https://api.fabric.microsoft.com/v1/workspaces/ admin','4498340c-27cf-4c6e-a025-00e5de6b0726')

# Determines whether lakehoues tables/shortcuts should be created before git sync due to warehouse dependencies 
has_wh_views_on_lh = True

# Set the Azure DevOps related parameters 
project_name=''
repo_name=''
main_branch = ''
branch_name = ''
org_name = ''
ado_api_url = 'https://dev.azure.com/'

# Azure Key Vault and Secret Name which stores the Azure DevOps PAT
key_vault_name = ''
secret_name = ''
target_capacity = '' # name or ID. leave empty '' to use the same capacity as source workspace


### Do not change these parameters  ####
# internal parameter to allow the installation of Python libraries when being run programatically. See https://learn.microsoft.com/en-us/fabric/data-engineering/library-management#python-inline-installation
_inlineInstallationEnabled = True

# indicates whether this script is called from Azure Devops
_runStandalone = True


##### Library imports and fabric rest client setup

https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric.fabricrestclient

In [ ]:
import pandas as pd
import datetime, time
import re,json, fnmatch,os
import requests, base64
import sempy
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException
from pyspark.sql import DataFrame
from pyspark.sql.functions import col,current_timestamp,lit
import sempy_labs as labs
from sempy_labs import migration, directlake
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model
from typing import Optional
from sempy_labs._helper_functions import (
    resolve_workspace_name_and_id,
    lro,
    _decode_b64,
)
import sempy_labs._icons as icons

import base64
from typing import Optional, Tuple, List
from uuid import UUID
import ast
from jsonpath_ng import jsonpath, parse

# instantiate the Fabric rest client
client = fabric.FabricRestClient()

# get the current workspace ID based on the context of where this notebook is run from
thisWsId = notebookutils.runtime.context['currentWorkspaceId']
thisWsName = notebookutils.runtime.context['currentWorkspaceName']

source_ws_id = fabric.resolve_workspace_id(source_ws)
target_ws_id = '' # will be set later

##### Always run this cell

Contains utility functions to support lakehouse and warehouse initialisation

and

Shortcut creator:
https://github.com/arasdk/fabric-code-samples/blob/main/shortcuts/fabric_shortcut_creator.py 

In [ ]:
#### 
### Setup and AzDO functions 
####


def _is_valid_uuid(
    guid: str,
):
    """
    Validates if a string is a valid GUID in version 4

    Parameters
    ----------
    guid : str
        GUID to be validated.

    Returns
    -------
    bool
        Boolean that indicates if the string is a GUID or not.
    """

    try:
        UUID(str(guid), version=4)
        return True
    except ValueError:
        return False

def get_capacity_status(p_target_cap):
    dfC = fabric.list_capacities()
    dfC_filt = dfC[dfC["Id"] == p_target_cap]
    return dfC_filt['State'].iloc[0]


#TODO create token via SPN
def get_branch_object_id(p_ado_url,p_branch_name, p_token):
    try:
        print(f"Retriving ID of main branch {branch_name} to be cloned ")
        headers = {'Authorization': f'Basic {p_token}',
                    'Content-Type': 'application/json'
                    }

        response = requests.get(f"{p_ado_url}/heads/{p_branch_name}?api-version=7.1", headers=headers)
        return response.json()["value"][0]["objectId"]
    except requests.exceptions.RequestException as e:
        print(f"Error getting branch object ID: {e}")
        return None

def encode_pat(pat):
    # Encode the PAT in base64
    encoded_pat = base64.b64encode(pat.encode('utf-8')).decode('utf-8')
    return encoded_pat

def create_azdo_branch(p_key_vault_name,p_secret_name, p_branch_name, p_main_branch,p_repo_name,p_ado_url):
    access_token =notebookutils.credentials.getToken('keyvault')
    url = f'https://{p_key_vault_name}.vault.azure.net/secrets/{p_secret_name}?api-version=7.3'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #print(response.json()['value'])
        pat_token =  encode_pat(':'+response.json()['value'])
    else:
        raise ValueError(f"Failed to get secret: {response.status_code} - {response.text}")

    try:
        print(f"Creating feature branch {p_branch_name} based on {p_main_branch} in progress")
        headers = {"Authorization": f"Basic {pat_token}", "Content-Type": "application/json"}
        data =  [
                    {
                "name":f"refs/heads/{p_branch_name}",
                "oldObjectId": "0000000000000000000000000000000000000000",
                "newObjectId": get_branch_object_id(p_ado_url,p_main_branch, pat_token)
                }
            ]
        response = requests.post(f"{p_ado_url}?api-version=7.1", headers=headers, json=data)
        response.raise_for_status()
        if (response.json()['value'][0]['success']):
            return True
        else:
            return False
    except requests.exceptions.RequestException as e:
        raise ValueError(f"Error creating Azure DevOps branch: {e}")



##### 
### Shortcut utility function 
####

# Extract workspace_id, item_id and path from a onelake URI
def extract_onelake_https_uri_components(uri):
    # Define a regular expression to match any string between slashes and capture the final path element(s) without the leading slash
    pattern = re.compile(r"abfss://([^@]+)@[^/]+/([^/]+)/(.*)")
    match = pattern.search(uri)
    if match:
        workspace_id, item_id, path = match.groups()
        return workspace_id, item_id, path
    else:
        return None, None, None


def is_valid_onelake_uri(uri: str) -> bool:
    workspace_id, item_id, path = extract_onelake_https_uri_components(uri)
    if "abfss://" not in uri or workspace_id is None or item_id is None or path is None:
        return False

    return True


def get_last_path_segment(uri: str):
    path = uri.split("/")  # Split the entire URI by '/'
    return path[-1] if path else None


def is_delta_table(uri: str):
    delta_log_path = os.path.join(uri, "_delta_log")
    return mssparkutils.fs.exists(delta_log_path)


def get_onelake_shorcut(workspace_id: str, item_id: str, path: str, name: str):
    shortcut_uri = (
        f"v1/workspaces/{workspace_id}/items/{item_id}/shortcuts/{path}/{name}"
    )
    result = client.get(shortcut_uri).json()
    return result


def is_folder_matching_pattern(path: str, folder_name: str, patterns: []):
    if folder_name in patterns:
        return True
    else:
        for pattern in patterns:
            if fnmatch.fnmatch(folder_name, pattern):
                return is_delta_table(path)

    return False


def get_matching_delta_tables_uris(uri: str, patterns: []) -> []:
    # Use a set to avoid duplicates
    matched_uris = set()
    files = mssparkutils.fs.ls(uri)
    folders = [item for item in files if item.isDir]

    # Filter folders to only those that matches the pattern and is a delta table
    matched_uris.update(
        folder.path
        for folder in folders
        if is_folder_matching_pattern(folder.path, folder.name, patterns)
    )

    return matched_uris


def create_onelake_shorcut(source_uri: str, dest_uri: str):
    src_workspace_id, src_item_id, src_path = extract_onelake_https_uri_components(
        source_uri
    )

    dest_workspace_id, dest_item_id, dest_path = extract_onelake_https_uri_components(
        dest_uri
    )

    name = get_last_path_segment(source_uri)
    dest_uri_joined = os.path.join(dest_uri, name)

    # If the destination path already exists, return without creating shortcut
    if mssparkutils.fs.exists(dest_uri_joined):
        print(f"Destination already exists: {dest_uri_joined}")
        return None

    request_body = {
        "name": name,
        "path": dest_path,
        "target": {
            "oneLake": {
                "itemId": src_item_id,
                "path": src_path,
                "workspaceId": src_workspace_id,
            }
        },
    }

    shortcut_uri = f"v1/workspaces/{dest_workspace_id}/items/{dest_item_id}/shortcuts"
    print(f"Creating shortcut: {shortcut_uri}/{name}..")
    try:
        client.post(shortcut_uri, json=request_body)
    except FabricHTTPException as e:
        print(e)
        return None

    return get_onelake_shorcut(dest_workspace_id, dest_item_id, dest_path, name)
   

####
## Copy lakehouse and warehouse utility functions
####

def get_lh_object_list(base_path,data_types = ['Tables', 'Files'])->pd.DataFrame:

    '''
    Function to get a list of tables for a lakehouse
    adapted from https://fabric.guru/getting-a-list-of-folders-and-delta-tables-in-the-fabric-lakehouse
    This function will return a pandas dataframe containing names and abfss paths of each folder for Files and Tables
    '''
    #data_types = ['Tables', 'Files'] #for if you want a list of files and tables
    #data_types = ['Tables'] #for if you want a list of tables

    df = pd.concat([
        pd.DataFrame({
            'name': [item.name for item in notebookutils.fs.ls(f'{base_path}/{data_type}/')],
            'type': data_type[:-1].lower() , 
            'src_path': [item.path for item in notebookutils.fs.ls(f'{base_path}/{data_type}/')],
        }) for data_type in data_types], ignore_index=True)

    return df

def get_wh_object_list(schema_list,base_path)->pd.DataFrame:

    '''
    Function to get a list of tables for a warehouse by schema
    '''
    data_type = 'Tables'
    dfs = []

    for schema_prefix in schema_list:
        if notebookutils.fs.exists(f'{base_path}/{data_type}/{schema_prefix}/'):
            items = notebookutils.fs.ls(f'{base_path}/{data_type}/{schema_prefix}/')
            if items:  # Check if the list is not empty
                df = pd.DataFrame({
                    'schema': schema_prefix,
                    'name': [item.name for item in items],
                    'type': data_type[:-1].lower(),
                    'src_path': [item.path for item in items],
                })
                dfs.append(df)

    if dfs:  # Check if the list of dataframes is not empty
        df = pd.concat(dfs, ignore_index=True)
    else:
        df = pd.DataFrame()  # Return an empty dataframe if no dataframes were created

    return df

def copy_lh_objects(table_list,workspace_src,workspace_tgt,lakehouse_src,lakehouse_tgt,fastcopy=True,usingIDs=False)->pd.DataFrame:
    # declare an array to keep the instrumentation
    cpresult = []
    # loop through all the tables to extract the source path 
    for table in table_list.src_path:
        source = table
        destination = source.replace(f'abfss://{workspace_src}', f'abfss://{workspace_tgt}')
        if usingIDs:
            destination = destination.replace(f'{lakehouse_src}', f'{lakehouse_tgt}')
        else:
            destination = destination.replace(f'{lakehouse_src}.Lakehouse', f'{lakehouse_tgt}.Lakehouse')
        start_time =  datetime.datetime.now()
        if notebookutils.fs.exists(destination):
             notebookutils.fs.rm(destination, True)
        if fastcopy:
            # use fastcopy util which is a python wrapper to azcopy
            notebookutils.fs.fastcp(source+'/*', destination+'/', True)
        else:
            notebookutils.fs.cp(source, destination, True)

        # recording the timing and add it to the results list
        end_time = datetime.datetime.now()
        copyreslist = [source, destination, start_time.strftime("%Y-%m-%d %H:%M:%S"),  end_time.strftime("%Y-%m-%d %H:%M:%S"), str((end_time - start_time).total_seconds())]
        cpresult.append(copyreslist)
    return pd.DataFrame(cpresult,columns =['source--------------------------------------','target--------------------------------------','start------------','end_time------------','elapsed seconds----'])

def createDWrecoverypl(ws_id,pl_name = 'Recover_Warehouse_Data_From_DR'):
  client = fabric.FabricRestClient()

  dfurl= "v1/workspaces/"+ ws_id + "/items"
  payload = { 
  "displayName": pl_name, 
  "type": "DataPipeline", 
  "definition": { 
    "parts": [ 
      { 
        "path": "pipeline-content.json", 
        "payload":  "ewogICAgInByb3BlcnRpZXMiOiB7CiAgICAgICAgImFjdGl2aXRpZXMiOiBbCiAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICJuYW1lIjogIkl0ZXJhdGVTY2hlbWFUYWJsZXMiLAogICAgICAgICAgICAgICAgInR5cGUiOiAiRm9yRWFjaCIsCiAgICAgICAgICAgICAgICAiZGVwZW5kc09uIjogW10sCiAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgIml0ZW1zIjogewogICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy50YWJsZXNUb0NvcHkiLAogICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgImJhdGNoQ291bnQiOiAyMCwKICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdGllcyI6IFsKICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiQ29weVdhcmVob3VzZVRhYmxlcyIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJDb3B5IiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdHkiOiAiU2V0IHRhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImRlcGVuZGVuY3lDb25kaXRpb25zIjogWwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIlN1Y2NlZWRlZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIF0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAicG9saWN5IjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0aW1lb3V0IjogIjAuMTI6MDA6MDAiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJyZXRyeSI6IDIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInJldHJ5SW50ZXJ2YWxJblNlY29uZHMiOiAzMCwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2VjdXJlT3V0cHV0IjogZmFsc2UsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZUlucHV0IjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNvdXJjZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZVNvdXJjZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJxdWVyeVRpbWVvdXQiOiAiMDI6MDA6MDAiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAicGFydGl0aW9uT3B0aW9uIjogIk5vbmUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGF0YXNldFNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFubm90YXRpb25zIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibGlua2VkU2VydmljZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICIwN2EwMzAwNl9kMWI2XzRhMzlfYmViMV8wYmJhMmFhZjVmZjciLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYW5ub3RhdGlvbnMiOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlUHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJlbmRwb2ludCI6ICJAcGlwZWxpbmUoKS5wYXJhbWV0ZXJzLmxha2Vob3VzZUNvbm5TdHIiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFydGlmYWN0SWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy5sYWtlaG91c2VJZCIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAid29ya3NwYWNlSWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy53b3Jrc3BhY2VJZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlVGFibGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidGFibGUiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6ICJAY29uY2F0KGNvbmNhdChpdGVtKCkuc2NoZW1hLCdfJyksaXRlbSgpLm5hbWUpIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRXhwcmVzc2lvbiIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzaW5rIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlU2luayIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJhbGxvd0NvcHlDb21tYW5kIjogdHJ1ZSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInRhYmxlT3B0aW9uIjogImF1dG9DcmVhdGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGF0YXNldFNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFubm90YXRpb25zIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibGlua2VkU2VydmljZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICIwYzAzMTIzYV9kMzEyXzQ2YzRfYThlN181YjRjYWQ4ZjEyZDciLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYW5ub3RhdGlvbnMiOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlUHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJlbmRwb2ludCI6ICJAcGlwZWxpbmUoKS5wYXJhbWV0ZXJzLndhcmVob3VzZUNvbm5TdHIiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFydGlmYWN0SWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy53YXJlaG91c2VJZCIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAid29ya3NwYWNlSWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy53b3Jrc3BhY2VJZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlVGFibGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidGFibGUiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6ICJAaXRlbSgpLm5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImVuYWJsZVN0YWdpbmciOiB0cnVlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0cmFuc2xhdG9yIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJUYWJ1bGFyVHJhbnNsYXRvciIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlQ29udmVyc2lvbiI6IHRydWUsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlQ29udmVyc2lvblNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFsbG93RGF0YVRydW5jYXRpb24iOiB0cnVlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInRyZWF0Qm9vbGVhbkFzTnVtYmVyIjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiU2V0IHRhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIlNldFZhcmlhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdHkiOiAiU2V0IHNjaGVtYSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRlbmN5Q29uZGl0aW9ucyI6IFsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJTdWNjZWVkZWQiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIF0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICBdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgInBvbGljeSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2VjdXJlT3V0cHV0IjogZmFsc2UsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZUlucHV0IjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhcmlhYmxlTmFtZSI6ICJUYWJsZW5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhbHVlIjogIkBpdGVtKCkubmFtZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIkV4cHJlc3Npb24iCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICJTZXQgc2NoZW1hIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIlNldFZhcmlhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwb2xpY3kiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZU91dHB1dCI6IGZhbHNlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzZWN1cmVJbnB1dCI6IGZhbHNlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YXJpYWJsZU5hbWUiOiAiU2NoZW1hbmFtZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhbHVlIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQGl0ZW0oKS5zY2hlbWEiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgIF0KICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgfQogICAgICAgIF0sCiAgICAgICAgInBhcmFtZXRlcnMiOiB7CiAgICAgICAgICAgICJsYWtlaG91c2VJZCI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjBmMGY2YjdjLTE3NjEtNDFlNi04OTZlLTMwMDE0ZjE2ZmY2ZCIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgInRhYmxlc1RvQ29weSI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogImFycmF5IiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiBbCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIkRhdGUiCiAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiR2VvZ3JhcGh5IgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIkhhY2tuZXlMaWNlbnNlIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIk1lZGFsbGlvbiIKICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6ICJkYm8iLAogICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICJUaW1lIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIlRyaXAiCiAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiV2VhdGhlciIKICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICBdCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJ3b3Jrc3BhY2VJZCI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjE1MDExNDNjLTI3MmYtNGEyZi05NzZhLTdlNTU5NzFlNGMyYiIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgIndhcmVob3VzZUlkIjogewogICAgICAgICAgICAgICAgInR5cGUiOiAic3RyaW5nIiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiAiNGQxYmQ5NTEtOTlkZS00YmQ3LWI3YmMtNzFjOGY1NmRiNDExIgogICAgICAgICAgICB9LAogICAgICAgICAgICAid2FyZWhvdXNlQ29ublN0ciI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjcyd3diaXZpMnViZWpicnRtdGFobzMyYjR5LWhxa2FjZmpwZTR4dXZmM2twemt6b2hzbWZtLmRhdGF3YXJlaG91c2UuZmFicmljLm1pY3Jvc29mdC5jb20iCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJsYWtlaG91c2VDb25uU3RyIjogewogICAgICAgICAgICAgICAgInR5cGUiOiAic3RyaW5nIiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiAiNzJ3d2JpdmkydWJlamJydG10YWhvMzJiNHktaHFrYWNmanBlNHh1dmYza3B6a3pvaHNtZm0uZGF0YXdhcmVob3VzZS5mYWJyaWMubWljcm9zb2Z0LmNvbSIKICAgICAgICAgICAgfQogICAgICAgIH0sCiAgICAgICAgInZhcmlhYmxlcyI6IHsKICAgICAgICAgICAgIlRhYmxlbmFtZSI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogIlN0cmluZyIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgIlNjaGVtYW5hbWUiOiB7CiAgICAgICAgICAgICAgICAidHlwZSI6ICJTdHJpbmciCiAgICAgICAgICAgIH0KICAgICAgICB9LAogICAgICAgICJsYXN0TW9kaWZpZWRCeU9iamVjdElkIjogIjRhYTIwYWY3LTk0YmQtNDM0OC1iZWY4LWY4Y2JjZDg0MGQ1MSIsCiAgICAgICAgImxhc3RQdWJsaXNoVGltZSI6ICIyMDI0LTExLTEzVDE1OjUyOjUyWiIKICAgIH0KfQ==", 
        "payloadType": "InlineBase64" 
      } 
    ] 
  } 
}   
  
  response = json.loads(client.post(dfurl,json= payload).content)
  return response['id']

def getItemId(wks_id,itm_name,itm_type):
    df = fabric.list_items(type=None,workspace=wks_id)
    #print(df)
    if df.empty:
        return 'NotExists'
    else:
        #display(df)
        #print(df.query('"Display Name"="'+itm_name+'"'))
        if itm_type != '':
            newdf= df.loc[(df['Display Name'] == itm_name) & (df['Type'] == itm_type)]['Id']
        else:
            newdf= df.loc[(df['Display Name'] == itm_name)]['Id']  
        if newdf.empty:
            return 'NotExists'
        else:
            return newdf.iloc[0]


# modified semantic link labs function - pendin PR
import sempy_labs._icons as icons
from typing import Optional, List
from sempy_labs._helper_functions import (
    resolve_workspace_name_and_id,
    _base_api,
)
from uuid import UUID

def initialize_git_connection(workspace: Optional[str | UUID] = None, initialization_strategy: Optional[str] = 'None') -> str:
    """
    Initializes a connection for a workspace that is connected to Git.

    This is a wrapper function for the following API: `Git - Initialize Connection <https://learn.microsoft.com/rest/api/fabric/core/git/initialize-connection>`_.

    Parameters
    ----------
    workspace : str | uuid.UUID, default=None
        The Fabric workspace name or ID.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    initialization_strategy : str
        Defines the strategy required for an initialization process when content 
        exists on both the remote side and the workspace side. 
        Additional strategies may be added over time.
        Supported options: PreferWorkspace, PreferRemote or None (default)

    Returns
    -------
    str
        Remote full SHA commit hash.
    """

    (workspace_name, workspace_id) = resolve_workspace_name_and_id(workspace)


    payload = {
        "initializationStrategy": initialization_strategy
    }
    response_json = _base_api(
        request=f"/v1/workspaces/{workspace_id}/git/initializeConnection",
        payload=payload,
        method="post",
        lro_return_json=True,
        status_codes=None,
    )
    
    print(
        f"{icons.green_dot} The '{workspace_name}' workspace git connection has been initialized."
    )
    return response_json.get("remoteCommitHash")



def initialize_git_connection_int(workspace: Optional[str | UUID] = None, initialization_strategy: Optional[str] = 'None') -> str:
    """
    Initializes a connection for a workspace that is connected to Git.

    This is a wrapper function for the following API: `Git - Initialize Connection <https://learn.microsoft.com/rest/api/fabric/core/git/initialize-connection>`_.

    Parameters
    ----------
    workspace : str | uuid.UUID, default=None
        The Fabric workspace name or ID.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    initialization_strategy : str
        Defines the strategy required for an initialization process when content 
        exists on both the remote side and the workspace side. 
        Additional strategies may be added over time.
        Supported options: PreferWorkspace, PreferRemote or None (default)

    Returns
    -------
    str
        Remote full SHA commit hash.
    """

    (workspace_name, workspace_id) = resolve_workspace_name_and_id(workspace)


    #payload = {
    #    "initializationStrategy": initialization_strategy
    #}
    url = f"https://wabi-west-us3-a-primary-redirect.analysis.windows.net/metadata/git/workspaces/{target_ws_id}/connection/initialize"
    #response_json = _base_api(
    #    request=f"/v1/workspaces/{workspace_id}/git/initializeConnection",
    #    payload=payload,
    #    method="post",
    #    lro_return_json=True,
    #    status_codes=None,
    #)
    from notebookutils import credentials

    data = {"mergePolicy":2}
    # Get the Fabric token
    token = credentials.getToken("pbi")

    # Print the token
    #print(token)
    access_token = token
    headers = {'content-type': 'application/json', 'authorization': f'Bearer {access_token}'}
    response = requests.post(url, headers=headers, json=data)

    print(
        f"{icons.green_dot} The '{workspace_name}' workspace git connection has been initialized."
    )
    return response.json().get("partialSyncBaseCommit")
    #return response_json.get("remoteCommitHash")


def create_shortcuts_or_copy_data(p_copy_lakehouse_data,p_create_lakehouse_shortcuts):
    df_lhs = labs.list_lakehouses(source_ws)
    for index, row in df_lhs.iterrows():


        if copy_lakehouse_data:
            lh_name= row['Lakehouse Name']
            if lh_name.find('temp')==-1:
                # Gathers the list of tables and source paths to be copied into the target lakehouse 
                src_path = f'abfss://{source_ws}@onelake.dfs.fabric.microsoft.com/{lh_name}.Lakehouse'

                table_list = get_lh_object_list(src_path)
                print(f'Attempting to copy table data for lakehouse {lh_name} from workspace {source_ws} to {target_ws}...')
                display(table_list)

                #print('Copy Lakehouse Delta tables...')
                res = copy_lh_objects(table_list[table_list['type']=='table'],source_ws,target_ws,
                                    lh_name,lh_name,False,False)
                display(res)
                # Copy files
                print(f'Attempting to copy file data for lakehouse {lh_name} from workspace {source_ws} to {target_ws}...')

                #print('Copy Lakehouse files...')
                res = copy_lh_objects(table_list[table_list['type']=='file'],source_ws,target_ws,
                                    lh_name,lh_name,False,False)
                display(res)
                print('Done.')

        else:
            # fetch ID of source lakehouse based on name and workspace
            source_lh_id = row['Lakehouse ID']
            target_lh_id = fabric.resolve_item_id(
                item_name=row['Lakehouse Name'], type="Lakehouse", workspace=target_ws
                )

            SOURCE_URI = f"abfss://{source_ws_id}@onelake.dfs.fabric.microsoft.com/{source_lh_id}/Tables"
            DEST_URI = f"abfss://{target_ws_id}@onelake.dfs.fabric.microsoft.com/{target_lh_id}/Tables"

            if PATTERN_MATCH is None or len(PATTERN_MATCH) == 0:
                raise TypeError("Argument 'PATTERN_MATCH' should be a valid list of patterns or ["*"] to match everything")

            # Collect created shortcuts
            result = []

            # If either URI's are invalid, just return
            if not is_valid_onelake_uri(SOURCE_URI) or not is_valid_onelake_uri(DEST_URI):
                print(
                    "invalid URI's provided. URI's should be in the form: abfss://<workspace-id>@onelake.dfs.fabric.microsoft.com/<item-id>/<path>"
                )
            else:
                # Remove any trailing '/' from uri's
                source_uri_addr = SOURCE_URI.rstrip("/")
                dest_uri_addr = DEST_URI.rstrip("/")

                dest_workspace_id, dest_item_id, dest_path = extract_onelake_https_uri_components(
                    dest_uri_addr
                )

                # If we are not shortcutting to a managed table folder or
                # the source uri is a delta table, just shortcut it 1-1.
                if not dest_path.startswith("Tables") or is_delta_table(source_uri_addr):
                    shortcut = create_onelake_shorcut(source_uri_addr, dest_uri_addr)
                    if shortcut is not None:
                        result.append(shortcut)
                else:
                    # If source is not a delta table, and destination is managed table folder:
                    # Iterate over source folders and create table shortcuts @ destination
                    for delta_table_uri in get_matching_delta_tables_uris(
                        source_uri_addr, PATTERN_MATCH
                    ):
                        shortcut = create_onelake_shorcut(delta_table_uri, dest_uri_addr)
                        if shortcut is not None:
                            result.append(shortcut)
            print(result)


########
### Pipeline utilities
########

def update_data_pipeline_definition(
    name: str, pipeline_content: dict, workspace: Optional[str] = None
):
    """
    Updates an existing data pipeline with a new definition.

    Parameters
    ----------
    name : str
        The name of the data pipeline.
    pipeline_content : dict
        The data pipeline content (not in Base64 format).
    workspace : str, default=None
        The name of the workspace.
        Defaults to None which resolves to the workspace of the attached lakehouse
        or if no lakehouse attached, resolves to the workspace of the notebook.
    """

    (workspace, workspace_id) = resolve_workspace_name_and_id(workspace)
    client = fabric.FabricRestClient()
    pipeline_payload = base64.b64encode(json.dumps(pipeline_content).encode('utf-8')).decode('utf-8')
    pipeline_id = fabric.resolve_item_id(
        item_name=name, type="DataPipeline", workspace=workspace
    )

    request_body = {
        "definition": {
            "parts": [
                {
                    "path": "pipeline-content.json",
                    "payload": pipeline_payload,
                    "payloadType": "InlineBase64"
                }
            ]
        }
    }


    response = client.post(
        f"v1/workspaces/{workspace_id}/items/{pipeline_id}/updateDefinition",
        json=request_body,
    )

    lro(client, response, return_status_code=True)

    print(
        f"{icons.green_dot} The '{name}' pipeline was updated within the '{workspace}' workspace."
    )

# Swaps the connection properties of an activity belonging to the specified item type(s)
def swap_pipeline_connection(pl_json: dict, p_source_ws: str,p_target_ws: str, 
                                p_item_type: List =['DataWarehouse','Lakehouse','Notebook'], 
                                p_conn_from_to: Optional[List[Tuple[str,str]]]=[]):
    
    source_ws_id = fabric.resolve_workspace_id(source_ws)

    target_ws_id = fabric.resolve_workspace_id(target_ws)

    if 'Warehouse' in p_item_type or 'Lakehouse' in p_item_type:
        ls_expr = parse('$..linkedService')
        for endpoint_match in ls_expr.find(pl_json):
            if endpoint_match.value['properties']['type'] == 'DataWarehouse' \
                and endpoint_match.value['properties']['typeProperties']['workspaceId'] == source_ws_id \
                and 'Warehouse' in p_item_type:
                # only update the warehouse if it was located in the source workspace i.e. we will update the properties to the target workspace if the warehouse resided in the same workspace as the pipeline
                warehouse_id = endpoint_match.value['properties']['typeProperties']['artifactId']
                warehouse_endpoint = endpoint_match.value['properties']['typeProperties']['endpoint']
                
                source_wh_name = fabric.resolve_item_name(item_id = warehouse_id,workspace=source_ws_id)
                # find the warehouse id of the warehouse with the same name in the target workspace
                target_wh_id = fabric.resolve_item_id(item_name = source_wh_name,type='Warehouse',workspace=target_ws_id)
                # look up the connection string for the warehouse in the target workspace
                whurl  = f"v1/workspaces/{target_ws_id}/warehouses/{target_wh_id}"
                whresponse = client.get(whurl)
                lhconnStr = whresponse.json()['properties']['connectionString']
                endpoint_match.value['properties']['typeProperties']['artifactId'] = target_wh_id
                endpoint_match.value['properties']['typeProperties']['workspaceId'] = target_ws_id
                endpoint_match.value['properties']['typeProperties']['endpoint'] = lhconnStr
                ls_expr.update(endpoint_match,endpoint_match.value)
            if endpoint_match.value['properties']['type'] == 'Lakehouse' \
                and endpoint_match.value['properties']['typeProperties']['workspaceId'] == source_ws_id \
                and 'Lakehouse' in p_item_type:
                #print(endpoint_match.value)
                lakehouse_id = endpoint_match.value['properties']['typeProperties']['artifactId']
                remote_lh_name = fabric.resolve_item_name(item_id = lakehouse_id,workspace=source_ws_id)
                # find the lakehouse id of the lakehouse with the same name in the target workspace
                target_lh_id = fabric.resolve_item_id(item_name = remote_lh_name,type='Lakehouse',workspace=target_ws_id)
                endpoint_match.value['properties']['typeProperties']['artifactId'] = target_lh_id
                endpoint_match.value['properties']['typeProperties']['workspaceId'] = target_ws_id
                ls_expr.update(endpoint_match,endpoint_match.value)
                #    print(endpoint_match.value)


    if 'Notebook' in p_item_type: 
        ls_expr = parse('$..activities')

        for endpoint_match in ls_expr.find(pl_json):
            for activity in endpoint_match.value:
                #print(activity['type'])
                if activity['type']=='TridentNotebook' and 'Notebook' in p_item_type: #only update if the notebook was in the same workspace as the pipeline
                    print('change from '+activity['typeProperties']['workspaceId'])
                    source_nb_id = activity['typeProperties']['notebookId']
                    source_nb_name = fabric.resolve_item_name(item_id = source_nb_id,workspace=source_ws_id)
                    target_nb_id = fabric.resolve_item_id(item_name = source_nb_name,type='Notebook',workspace=target_ws_id)
                    activity['typeProperties']['notebookId']=target_nb_id
                    activity['typeProperties']['workspaceId']=target_ws_id
                    print('to notebook '+ target_nb_id)
                    #ls_expr.update(endpoint_match,endpoint_match.value)

    if p_conn_from_to:
        for ti_conn_from_to in p_conn_from_to:
            if not _is_valid_uuid(ti_conn_from_to[0]):
                print('Connection from is string '+ str(ti_conn_from_to[0]))
                dfC_filt = df_conns[df_conns["Connection Name"] == ti_conn_from_to[0]]       
                connId_from = dfC_filt['Connection Id'].iloc[0]     
            else:
                connId_from = ti_conn_from_to[0]

            if not _is_valid_uuid(ti_conn_from_to[1]):
                print('Connection from is string '+ str(ti_conn_from_to[1]))
                dfC_filt = df_conns[df_conns["Connection Name"] == ti_conn_from_to[1]]       
                connId_to = dfC_filt['Connection Id'].iloc[0]     
            else:
                connId_to = ti_conn_from_to[1]

            ls_expr = parse('$..externalReferences')
            for externalRef in ls_expr.find(pl_json):
                if externalRef.value['connection']==connId_from:
                    print('Changing connection from '+str(connId_from))
                    externalRef.value['connection']=connId_to
                    ls_expr.update(externalRef,externalRef.value)
                    print('to '+str(connId_to))

    return pl_json



##### Standalone mode: Create AzDO branch and workspace

This cell runs only if not invoked from AzDO pipeline

In [ ]:
if _runStandalone:
    # get capacity details
    if target_capacity != '':
        if _is_valid_uuid(target_capacity):
            target_capacity_id = target_capacity.lower()
        else: 
            target_capacity_id = labs.resolve_capacity_id(target_capacity)
    else: # if not set then fetch capacity of source workspace
        target_capacity_id = labs.resolve_workspace_capacity(source_ws)[0]
        target_capacity = labs.resolve_capacity_name(target_capacity_id)
    # check capacity status
    cap_status = get_capacity_status(target_capacity_id)
    if cap_status == 'Inactive':
        raise ValueError(f"Status of capacity {target_capacity} is {cap_status}. Please resume the capacity and retry")
    else:
        print(f"Status of capacity {target_capacity} is {cap_status}")



   # Create Azure DevOps Branch
    ado_api_url = f"{ado_api_url}/{org_name}/{project_name}/_apis/git/repositories/{repo_name}/refs"
    result = create_azdo_branch(key_vault_name,secret_name, branch_name, main_branch,repo_name, ado_api_url)
    if result:
        print(f"Feature branch {branch_name} created")
    else:
        raise ValueError(f"Could not create branch {branch_name}, perhaps it already exists?")

    # Create new feature workspace
    try:
        print(f"Creating workspace: " + target_ws + " in capacity "+ target_capacity_id +"...",end="")
        target_ws_id =  fabric.create_workspace(target_ws,target_capacity_id) 
        print(f"done. New workspace ID = {target_ws_id = }")
    except Exception as error:
        errmsg =  f"Failed to create workspace {target_ws} with capacity ID ({target_capacity_id}) due to: {str(error)}"
        raise ValueError(errmsg)


##### If required, create Lakehouses in new feature workspace

This cell will only run if there are warehouse views which depend on lakehouse tables. Determined by the parameter has_wh_views_on_lh set to True above.
<br>In this case the lakehouses will be pre-created. Shortcuts created or data copied done in subsequent cell

In [ ]:
if has_wh_views_on_lh:
    lhs = fabric.list_items(type='Lakehouse',workspace=source_ws)
    for index,row in fabric.list_items(type='Lakehouse',workspace=source_ws).iterrows():
        #try:
        print(f"Creating {row['Display Name']} lakehouse...",end="")
        fabric.create_lakehouse(display_name = row['Display Name'],description='Created programatically via branch out script',max_attempts=3, workspace=target_ws)
        print('done')

##### Either create shortcuts from source to target lakehouse(s) or copy data

Loops through lakehouse(s) in the target workspace and either populates them with shortcuts or data



In [ ]:
# ensure target workspace ID is set based on name
if target_ws_id == '':
    target_ws_id = fabric.resolve_workspace_id(target_ws)

# populate the lakehouses with shortcuts or data
create_shortcuts_or_copy_data(copy_lakehouse_data,create_lakehouse_shortcuts)
time.sleep(120) # wait for sql endpoint to update


##### If not done in ADO, connect workspace to Git, initialize and update
This cell will only run if there are warehouse views which depend on lakehouse tables based on parameter has_wh_views_on_lh

In [ ]:
if has_wh_views_on_lh:

    # branch name should be the same name as the target workspace, if not modify parameters to pass in the branch name from ADO
    branch_name = target_ws 
    # fetch existing Git connection details from source workspace
    gitconnx = labs.get_git_connection(source_ws)
    # connect target workspace to the same git provider and new branch
    if gitconnx.loc[0, 'Git Provider Type'] == 'AzureDevOps':
        result = labs.connect_workspace_to_azure_dev_ops(
        gitconnx.loc[0, 'Organization Name'],
        gitconnx.loc[0, 'Project Name'],
        gitconnx.loc[0, 'Repository Name'],
        branch_name,
        gitconnx.loc[0, 'Directory Name'],
        target_ws)
    elif gitconnx.loc[0, 'Git Provider Type'] == 'GitHub':  #### TODO ####
        #TODO connect workspace with Github details
        labs.connect_workspace_to_github(
            '', #owner_name: str,
                gitconnx.loc[0, 'Repository Name'], #repository_name: str,
            branch_name,
            gitconnx.loc[0, 'Directory Name'], #directory_name: str,
            '', #connection_id: UUID,
            '', #source: str = "ConfiguredConnection",
            target_ws)
    else:
        raise ValueError(f"Unsupported Git provider type {gitconnx.loc[0, 'Git Provider Type']}.")

In [ ]:
if has_wh_views_on_lh:
    # inialize the git connection
    commit_hash = initialize_git_connection(target_ws,'PreferRemote')


    # update from git 
    labs.update_from_git(remote_commit_hash=commit_hash,
                        conflict_resolution_policy='PreferRemote', 
                        allow_override=True,
                        workspace=target_ws)



In [ ]:
labs.commit_to_git(comment='Initial',  workspace=target_ws)

##### Update default and attached lakehouses/warehouses for notebooks

Update notebook dependencies based on but now supports T-SQL notebooks:
https://github.com/PowerBiDevCamp/FabConWorkshopSweden/blob/main/DemoFiles/GitUpdateWorkspace/updateWorkspaceDependencies_v1.ipynb


In [ ]:
for notebook in notebookutils.notebook.list(workspaceId=target_ws_id):
    updates = False
    if notebook.displayName == 'ETL':#True: #notebook.displayName == 'T-SQL_Notebook': #notebook.displayName != 'Create Feature Branch':

        # Get the current notebook definition
        json_payload = json.loads(notebookutils.notebook.getDefinition(notebook.displayName,workspaceId=source_ws_id))
        #print(json.dumps(json_payload, indent=4))
        # Check for any attached lakehouses
        if 'dependencies' in json_payload['metadata'] \
            and 'lakehouse' in json_payload['metadata']['dependencies'] \
            and json_payload['metadata']["dependencies"]["lakehouse"] is not None:
            # Extract attached and default lakehouses
            current_lakehouse = json_payload['metadata']['dependencies']['lakehouse']
            # if default lakehouse setting exists
            if 'default_lakehouse_name' in current_lakehouse:
                print(f"Updating notebook {notebook.displayName} with new default lakehouse: {current_lakehouse['default_lakehouse_name']} in workspace {target_ws}")
                source_lh_name = fabric.resolve_item_name(item_id = current_lakehouse['default_lakehouse'],type='Lakehouse',workspace=source_ws_id)
                current_lakehouse['default_lakehouse'] = fabric.resolve_item_id(item_name = source_lh_name,type='Lakehouse',workspace=target_ws_id)
                current_lakehouse['default_lakehouse_workspace_id'] = target_ws_id
                updates = True
            # loop through all attached lakehouess
            for lakehouse in json_payload['metadata']['dependencies']['lakehouse']['known_lakehouses']:
                source_lh_id = lakehouse['id']
                # find source lakehouse name
                source_lh_name = fabric.resolve_item_name(item_id = lakehouse['id'],type='Lakehouse',workspace=source_ws_id)
                # find target lakehouse id based on name
                target_lh_id = fabric.resolve_item_id(item_name = source_lh_name,type='Lakehouse',workspace=target_ws_id)
                lakehouse['id'] = target_lh_id
                print(f'Updating attached lakehouse {source_lh_name} from {source_lh_id} to target ID {target_lh_id}')
                updates = True

        if 'dependencies' in json_payload['metadata'] and 'warehouse' in json_payload['metadata']['dependencies']:
            # Fetch existing details
            current_warehouse = json_payload['metadata']['dependencies']['warehouse']
            current_warehouse_id = current_warehouse['default_warehouse']
            source_wh_name =  fabric.resolve_item_name(item_id = current_warehouse_id,workspace=source_ws_id)
            #print('Source warehouse name is ' + source_wh_name)
            target_wh_id = fabric.resolve_item_id(item_name = source_wh_name,type='Warehouse',workspace=target_ws_id)

            if 'default_warehouse' in current_warehouse:
                #json_payload['metadata']['dependencies']['warehouse'] = {}
                print(f"Attempting to update notebook {notebook.displayName} with new default warehouse: {target_wh_id} in {target_ws}")
            
                json_payload['metadata']['dependencies']['warehouse']['default_warehouse'] = target_wh_id
                for warehouse in json_payload['metadata']['dependencies']['warehouse']['known_warehouses']:
                    if warehouse['id'] == current_warehouse_id:
                        warehouse['id'] = target_wh_id
                        updates = True

        if updates:
            notebookutils.notebook.updateDefinition(
                    name = notebook.displayName,
                    content  = json.dumps(json_payload),
                    workspaceId = target_ws_id
                    )
            
            print(f"Updated notebook {notebook.displayName} in {target_ws}")

        else:
            print(f'No default lakehouse set for notebook {notebook.displayName}, ignoring.')

##### Copy warehouse data via parameterised pipeline

Loop through all warehouses and copy the data

In [ ]:
if copy_warehouse_data:
    p_logging_verbose = False
    df_warehouses = (labs.list_warehouses(target_ws))
    #display(df_warehouses)
    for index, row in df_warehouses.iterrows():
        source_wh_id = labs.resolve_warehouse_id(row['Warehouse Name'],source_ws_id)
        target_wh_id = labs.resolve_warehouse_id(row['Warehouse Name'],target_ws_id)
        
        src_path = f'abfss://'+source_ws_id+'@onelake.dfs.fabric.microsoft.com/'+source_wh_id
        tgt_path = f'abfss://'+target_ws_id+'@onelake.dfs.fabric.microsoft.com/'+target_wh_id

        # extract the list of schemas per data 
        schema_list = get_lh_object_list(src_path,['Tables'])
        # extract a list of warehouse objects per schema and store in a list
        table_list = get_wh_object_list(schema_list['name'],src_path)
    
        # create a temporary staging lakehouse per warehouse to create shortcuts into, 
        # which point back to original warehouse data currently in the DR storage account
        lhname = 'temp_rlh_' + source_ws+'_'+row['Warehouse Name']
        # check if it exists before attempting create
        if p_logging_verbose:
            print('Checking whether the temporary lakehouse "'+ lhname +'" exists in workspace '+target_ws+'...')
        temp_lh_id = getItemId(target_ws_id,lhname,'Lakehouse')
        if temp_lh_id == 'NotExists':
            lhname = lhname[:256] # lakehouse name should not exceed 256 characters
            payload = payload = '{"displayName": "' + lhname + '",' \
            + '"description":  "Interim staging lakehouse for primary warehouse recovery: ' \
            + source_ws+'_'+row['Warehouse Name'] + 'into workspace '+ target_ws + '(' + target_ws +')"}'
            try:
                lhurl = "v1/workspaces/" + target_ws_id + "/lakehouses"
                lhresponse = client.post(lhurl,json= json.loads(payload))
                temp_lh_id = lhresponse.json()['id']
                if p_logging_verbose:
                    print('Temporary lakehouse "'+ lhname +'" created with Id ' + temp_lh_id + ': ' + str(lhresponse.status_code) + ' ' + str(lhresponse.text))
            except Exception as error:
                print(error.errorCode)
        else:
            if p_logging_verbose:
                print('Temporary lakehouse '+lhname+' (' + temp_lh_id + ') already exists.')
            
        time.sleep(60) # waiting for temporary lakehouse to provision completely   

        # Create shortcuts for every table in the format of schema_table under the tables folder
        for index,itable in table_list.iterrows():
            shortcutExists=False
            # Check if shortcut exists
            try:
                url = "v1/workspaces/" + target_ws_id + "/items/" + temp_lh_id + "/shortcuts/Tables/"+itable['schema']+'_'+itable['name']
                tlhresponse = client.get(url)
                shortcutExists = True
                if p_logging_verbose:
                    print('Shortcut '+itable['schema']+'_'+itable['name'] +' already exists')
            except Exception as error:
                shortcutExists = False    

            if not shortcutExists: 
                # Create shortcuts - one per table per schema
                url = "v1/workspaces/" + target_ws_id + "/items/" + temp_lh_id + "/shortcuts"
                scpayload = '{' \
                '"path": "Tables/",' \
                '"name": "'+itable['schema']+'_'+itable['name']+'",' \
                '"target": {' \
                '"oneLake": {' \
                    '"workspaceId": "' + source_ws_id + '",' \
                    '"itemId": "'+ source_wh_id +'",' \
                    '"path": "/Tables/' + itable['schema']+'/'+itable['name'] + '"' \
                    '}}}' 
                try:
                    #print(scpayload)                
                    shctresponse = client.post(url,json= json.loads(scpayload))
                    if p_logging_verbose:
                        print('Shortcut '+itable['schema']+'_'+itable['name'] + ' created.' )

                except Exception as error:
                    print('Error creating shortcut '+itable['schema']+'_'+itable['name']+' due to '+str(error) + ':' + shctresponse.text)
        
        recovery_pipeline_prefix= 'plRecover_WH'       
        # recovery pipeline name should not exceed 256 characters
        recovery_pipeline = recovery_pipeline_prefix+'_'+source_ws + '_'+row['Warehouse Name'][:256]
        if p_logging_verbose:
            print('Attempting to deploy a copy pipeline in the target workspace to load the target warehouse tables from the shortcuts created above... ')
        # Create the pipeline in the target workspace that loads the target warehouse from shortcuts created above 
        plid = getItemId( target_ws_id,recovery_pipeline,'DataPipeline')
        #print(plid)
        if plid == 'NotExists':
        plid = createDWrecoverypl(target_ws_id,recovery_pipeline_prefix+'_'+source_ws + '_'+row['Warehouse Name'])
        if p_logging_verbose:
            print('Recovery pipeline ' + recovery_pipeline + ' created with Id '+plid)
        else:
        if p_logging_verbose:
            print('Datawarehouse recovery pipeline "' + recovery_pipeline + '" ('+plid+') already exist in workspace "'+target_ws + '" ('+target_ws_id+')')  
            print('\n')

        tablesToCopyParam = table_list[['schema','name']].to_json( orient='records')
        # ensure the temporary lakehouse exists

        # obtain the connection string for the lakehouse to pass to the copy pipeline
        whurl  = "v1/workspaces/" + target_ws_id + "/lakehouses/" + temp_lh_id
        whresponse = client.get(whurl)
        lhconnStr = whresponse.json()['properties']['sqlEndpointProperties']['connectionString']

        # get the SQLEndpoint ID of the lakehouse to pass to the copy pipeline
        items = fabric.list_items(workspace=target_ws_id)
        #print(items)
        temp_lh_sqle_id = items[(items['Type'] == 'SQLEndpoint') & (items['Display Name']==lhname)]['Id'].values[0]


        # obtain the connection string for the warehouse to pass to the copy pipeline    
        whurl  = "v1/workspaces/" + target_ws_id + "/warehouses/" + target_wh_id
        whresponse = client.get(whurl)
        whconnStr = whresponse.json()['properties']['connectionInfo']

        # obtain the pipeline id created to recover this warehouse
        plid = getItemId( target_ws_id,recovery_pipeline,'DataPipeline')
        if plid == 'NotExists':
            print('Error: Could not execute pipeline '+recovery_pipeline+ ' as the ID could not be obtained ')
        else:
            # pipeline url including pipeline Id unique to each warehouse
            plurl = 'v1/workspaces/'+target_ws_id+'/items/'+plid+'/jobs/instances?jobType=Pipeline'
            #print(plurl)

            payload_data = '{' \
                '"executionData": {' \
                    '"parameters": {' \
                        '"lakehouseId": "' + temp_lh_sqle_id + '",' \
                        '"tablesToCopy": ' + tablesToCopyParam + ',' \
                        '"workspaceId": "' + target_ws_id +'",' \
                        '"warehouseId": "' + target_wh_id + '",' \
                        '"lakehouseConnStr": "' + lhconnStr + '",' \
                        '"warehouseConnStr": "' + whconnStr + '"' \
                        '}}}'
            #print(payload_data)
            plresponse = client.post(plurl, json=json.loads(payload_data))
            if p_logging_verbose:
                print(str(plresponse.status_code))      
    print('Done')


##### Update directlake model lakehouse/warehouse connection

https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.update_direct_lake_model_connection    

In [ ]:

df_datasets = fabric.list_datasets(target_ws)

# Iterate over each dataset in the dataframe
for index, row in df_datasets.iterrows():
    try:
        # Check if the dataset is not the default semantic model
        if not labs.is_default_semantic_model(row['Dataset Name'], fabric.resolve_workspace_id(target_ws)):
            #print('Updating semantic model connection ' + row['Dataset Name'] + ' in workspace '+ target_ws)
            labs.directlake.update_direct_lake_model_connection(dataset=row['Dataset Name'], 
                                                                            workspace= target_ws,
                                                                            source=labs.directlake.get_direct_lake_source(row['Dataset Name'], workspace= target_ws)[1], 
                                                                            source_type=labs.directlake.get_direct_lake_source(row['Dataset Name'], workspace= target_ws)[0], 
                                                                            source_workspace=target_ws)
            labs.refresh_semantic_model(dataset=row['Dataset Name'], workspace= target_ws)
    except Exception as error:
        errmsg =  f"Failed to update and refresh semantic model {row['Dataset Name']} due to: {str(error)}"
        print(errmsg)
        #raise ValueError(errmsg)


##### Rebind reports to local datasets

https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.report.html#sempy_labs.report.report_rebind

In [ ]:
df_reports = fabric.list_reports(workspace=target_ws)
for index, row in df_reports.iterrows():
    #print(row['Name'] + '-' + row['Dataset Id'])
    df_datasets = fabric.list_datasets(workspace=target_ws)
    dataset_name = df_datasets[df_datasets['Dataset ID'] == row['Dataset Id']]['Dataset Name'].values[0]
    print(f'Rebinding report to {dataset_name} in {target_ws}')
    labs.report.report_rebind(report=row['Name'],dataset=dataset_name, report_workspace=target_ws, dataset_workspace=target_ws)


##### Update data pipeline source & sink connections

Support changes lakehouses, warehouses, notebooks and connections from source to target. <br>
Connections changes should be expressed as an array of tuples [{from_1:to_1},{from_N:to_N}]

In [ ]:
if len(connections_from_to)>0: 
    # convert from a string to a proper type i.e. list of tuples 
    #connections_from_to = ast.literal_eval(connections_from_to)
    # loading a dataframe of connections to perform an ID lookup if required 
    df_conns = labs.list_connections()

    df_pipeline = labs.list_data_pipelines(source_ws)
    for index, row in df_pipeline.iterrows():
        pipeline_json = json.loads(labs.get_data_pipeline_definition(row['Data Pipeline Name'],source_ws))

        p_new_json = swap_pipeline_connection(pipeline_json, source_ws,target_ws,
                ['DataWarehouse','Lakehouse','Notebook'],
                [connections_from_to]) 
        #print(json.dumps(pipeline_json, indent=4))
        
        update_data_pipeline_definition(name=row['Data Pipeline Name'],pipeline_content=pipeline_json, workspace=target_ws)


##### Commit changes made above to Git

In [ ]:
labs.commit_to_git(comment='Initial',  workspace=target_ws)